<a href="https://colab.research.google.com/github/injoon-pij/pytorch-learning/blob/master/pytorch_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) Text Classification

텍스트 분류는 RNN의 다-대-일(Many-to-One) 문제에 속함. 
* 모든 시점(time step)에 대해서 입력을 받지만 최종 시점의 RNN 셀만이 은닉 상태를 출력하고, 이것이 출력층으로 가서 활성화 함수를 통해 정답을 고르는 문제


텍스트 분류 관점에서 앞서 배운 RNN 코드의 timesteps와 input_dim, 그리고 hidden_size를 해석해보면 다음과 같다

```python
nn.RNN(input_size, hidden_size, batch_first=True)
```

* hidden_size = 출력의 크기(output_dim).
* timesteps = 시점의 수 = 각 문서에서의 단어 수.
* input_size = 입력의 크기 = 각 단어의 벡터 표현의 차원 수.

# 2) IMDB Movie Review Sentiment Analysis

IMDB의 리뷰 데이터는 리뷰에 대한 텍스트와 해당 리뷰가 긍정인 경우 1을 부정인 경우 0으로 표시한 레이블로 구성된 데이터

파이토치에서는 해당 IMDB 영화 리뷰 데이터를 바로 다운로드 할 수 있도록 지원하고 있다

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext.legacy import data, datasets
import random

# seed
SEED = 5
random.seed(SEED)
torch.manual_seed(SEED)

# hyperparameter
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 10

# cpu/gpu
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("cpu와 cuda 중 다음 기기로 학습함:", DEVICE)

cpu와 cuda 중 다음 기기로 학습함: cuda


## 2.1 Data preprocessing

### 2.1.1 Data Field

* LABEL
 * use_vocab : dataset.IMDB의 레이블 데이터는 'neg', 'pos' 형태, 즉 텍스트 형태로 되어있으므로 데이터 학습 시 레이블을 정수 인덱스로 바꾸기 위해 단어 집합 딕셔너리를 만들 것임 

In [2]:
# 필드 정의
TEXT = data.Field(sequential=True,
                  lower = True,
                  batch_first = True)

LABEL = data.Field(sequential= False,
                   batch_first = True,
                   is_target = True)

### 2.1.2 Data Load


torchtext.legacy.datasets을 통해 IMDB 리뷰 데이터를 다운로드할 수 있다

In [3]:
# 데이터를 다운받는 동시에 훈련 데이터와 테스트 데이터를 분할해 dataset 형태로 저장
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:05<00:00, 15.3MB/s]


In [4]:
print(len(trainset))
print(len(testset))

25000
25000


In [5]:
print(vars(trainset[0]))

{'text': ['in', 'terms', 'of', 'the', 'arts,', 'the', '1970s', 'were', 'a', 'very', 'turbulent', 'era.', 'in', 'literature', 'and', 'the', 'visual', 'arts,', 'it', 'was', 'the', 'closing', 'of', 'a', 'great', 'fifty', 'or', 'sixty', 'year', 'period', 'of', 'creativity', 'that', 'has', 'yet', 'to', 'be', 'restarted.', 'in', 'music', 'it', 'was', 'a', 'decade', 'that', 'many', 'see', 'as', 'a', 'low', 'point,', 'due', 'to', 'corporate', 'rock', 'and', 'disco.', 'on', 'television', 'it', 'was', 'a', 'golden', 'age', 'for', 'situation', 'comedies,', 'from', 'the', 'odd', 'couple', 'to', 'the', 'mary', 'tyler', 'moore', 'show', 'to', 'm*a*s*h', 'to', 'all', 'in', 'the', 'family,', 'but', 'in', 'film', 'it', 'was', 'even', 'a', 'greater', 'period', 'of', 'creativity,', 'in', 'all', 'genres,', 'that', 'saw', 'the', 'rise', 'of', 'the', 'american', 'auteur-', 'directors', 'like', 'robert', 'altman,', 'francis', 'ford', 'coppola,', 'and', 'martin', 'scorsese-', 'from', 'the', 'ashes', 'of', 'th

### 2.1.3 Vocabulary

In [6]:
# 단어 집합 생성
TEXT.build_vocab(trainset, min_freq = 5)
LABEL.build_vocab(trainset)

In [7]:
LABEL.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.legacy.vocab.Vocab object at 0x7f2777f84250>>,
            {'<unk>': 0, 'neg': 1, 'pos': 2})

* dataset.IMDB의 레이블 데이터는 'neg', 'pos' 형태로, 즉 텍스트 형태로 되어있으므로 추후 학습 시 정답 레이블을 정수 인덱스로 바꾸기 위해 단어 집합 딕셔너리를 만듬

In [8]:
vocab_size = len(TEXT.vocab)
n_classes = 2
print('단어 집합의 크기 : {}'.format(vocab_size))
print('클래스의 개수 : {}'.format(n_classes))

단어 집합의 크기 : 46159
클래스의 개수 : 2


### 2.1.4 DataLoader

In [9]:
# 모델 평가 위한 검증 데이터 분할
trainset, valset = trainset.split(split_ratio=0.8)

* dataset.split : 비율에 따라 데이터셋 분할

In [10]:
train_iter, val_iter, test_iter = data.BucketIterator.splits(
                                            (trainset, valset, testset), 
                                            batch_size=BATCH_SIZE,
                                            shuffle=True, 
                                            repeat=False)

* BucketIterator(tuple of datasets) : Defines an iterator that __batches examples of similar lengths together__
 * 일반 Iterator와 다른 점은, 입력 데이터의 시퀀스 길이가 서로 다른 경우에 서로 길이가 비슷한 데이터들끼리 모아 미니배치를 구성하게 만든다는 것임
 * splits : 여러 dataset으로부터 각각의 Iterator 생성

* repeat = False : 미니배치 연산자들이 한 에폭(dataset의 처음부터 끝)을 다 끝내면 iterate를 멈춤
 * repeat = True : 에폭이 끝나도 연산자 반복이 무한히 진행되므로, 에폭 단위가 아닌 iteration 단위로 성능을 측정하고자 할 때 주로 사용. 이때 인위적으로 iteration을 멈춰주지 않으면 무한히 반복하므로 주의.

In [11]:
print('훈련 데이터의 미니 배치의 개수 : {}'.format(len(train_iter)))
print('테스트 데이터의 미니 배치의 개수 : {}'.format(len(test_iter)))
print('검증 데이터의 미니 배치의 개수 : {}'.format(len(val_iter)))

훈련 데이터의 미니 배치의 개수 : 313
테스트 데이터의 미니 배치의 개수 : 391
검증 데이터의 미니 배치의 개수 : 79


### 2.1.5 Modeling(GRU)

GRU는 LSTM과 같은 은닉층에 게이트가 추가된 RNN 모델 중 하나이다

In [12]:
class GRU(nn.Module):

    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(GRU, self).__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):

        # (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        x = self.embed(x)

        # 최초 은닉 상태값를 0벡터로 초기화
        h_0 = self._init_state(batch_size=x.size(0))

        # x.shape() => (배치 크기, 시퀀스 길이, 은닉 상태의 크기)
        x, _ = self.gru(x, h_0)

        # 마지막 time-step의 은닉 상태만 가져옴
        # (배치 크기, 시퀀스 길이, 은닉 상태의 크기) => (배치 크기, 은닉 상태의 크기)
        h_t = x[:,-1,:]

        self.dropout(h_t)

        # (배치 크기, 은닉 상태의 크기) -> (배치 크기, 출력층의 크기)
        logit = self.out(h_t)

        return logit

    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new_zeros(self.n_layers, batch_size, self.hidden_dim)

* new_zeros() : 입력된 사이즈와 같은 형태로 새로운 제로 텐서를 반환
 * 입력 사이즈 형태는 tuple 이나 list

* 필요한 크기의 제로텐서를 새로 정의하지 않고 굳이 모델 내부의 파라미터를 new_zeros()로 받아 다시 만든 이유는 new() 함수가 반환하는 새로 구성된 텐서는 __기존 텐서와 동일한 dtype, device를 유지한 채로 반환되기 때문__
 * 모델의 device를 GPU로 정의해도 모델 내부에서 새로운 텐서를 따로 정의하면 해당 텐서는 device가 CPU로 정의되기 때문에, GPU로 정의된 모델 내부의 파라미터를 기반으로 new() 함수 사용한 것 
 * 또한 class 내부에 텐서를 새로 정의하려면 class 내부에 torch를 불필요하게 import 해야 하므로 이를 지양하고, 이미 존재하는 텐서를 사용하기 위한 것

```python
# new_zeros()
tensor = torch.tensor((), dtype=torch.float64)
tensor
>>> tensor([], dtype=torch.float64)

tensor.new_zeros((2, 3))
>>> tensor([[ 0.,  0.,  0.],
            [ 0.,  0.,  0.]], dtype=torch.float64)

tensor.new_zeros([5,6])
>>> tensor([[0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0.]], dtype=torch.float64)
```

In [13]:
model = GRU(n_layers = 1, hidden_dim = 256, n_vocab = vocab_size, embed_dim = 128, n_classes = n_classes, dropout_p = 0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# DEVICE = cuda인 경우, model의 parameter도 cuda 연산인지 확인
# next(model.parameters()).data.device
# >>> device(type='cuda', index=0)

In [14]:
def train(model, optimizer, train_iter):
    model.train() # tells your model that you are training the model
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)  # 레이블 값을 0과 1로 변환
        optimizer.zero_grad()

        logit = model(x)

        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()

* ```y.data.sub_(1)``` : 현재 레이블값이 'neg' = 1, 'pos' = 2 형태로 되어있으므로 긍/부정 이중분류를 적용하기 위해 레이블값을 0과 1로 변환해줌

In [18]:
def evaluate(model, val_iter):
    """evaluate model"""
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1) # 레이블 값을 0과 1로 변환

        logit = model(x)

        loss = F.cross_entropy(logit, y, reduction='sum')
        total_loss += loss.item() # .item() : 손실이 갖고 있는 스칼라 값을 가져옴 
        corrects += (logit.max(dim = 1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy

* ```with torch.no_grad()``` vs. ```model.eval()```

 * ```with torch.no_grad()``` : 모델 평가(추론) 시에 명령어들을 ```with torch.no_grad()```에 포함시키게 되면 Pytorch는 autograd engine을 꺼버린다. 이 말은 더 이상 자동으로 gradient를 트래킹하지 않는다는 말이다. 물론 추론 과정에서 loss.backward()를 통해 역전파를 진행하지 않는다면 굳이 ```torch.no_grad()```에 포함시키지 않아도 gradient를 계산하지 않을 것이지만, __torch.no_grad()의 주된 목적은 autograd를 끔으로써 메모리 사용량을 줄이고 연산 속도를 높히기 위함이다.__ 그래서 일반적으로 inference를 진행할 때는 ```with torch.no_grad()``` 로 명령어로 감싼다.

 * ```model.eval()``` : ```model.eval()```의 역할은 ```with torch.no_grad()```와 약간 다르다. 학습 시에는 동작해야하지만, 추론 시에는 동작하지 않는 Dropout layer나 BatchNorm layer처럼 모델링 시 상황에 따라 다르게 동작하는 layer들이 존재한다. ```model.eval()```는 이런 layer들의 동작을 추론 상황으로 바꿔준다는 목적으로 사용된다. 
 
 * 따라서, 우리가 보통 원하는 모델의 동작을 위해서는 위의 두 가지를 모두 사용해야하는 것이 맞다.

* ```cross_entropy(... reduction = 'sum')```
 * reduction (string, optional) – Specifies the reduction to apply to the output
   * 'none': no reduction will be applied
   * 'mean': the sum of the output will be divided by the number of elements in the output
   * 'sum': the output will be summed

* ```corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()```
 * .view(y.size()) : view로 출력값의 사이즈를 y의 사이즈와 같게 조정 (사실 위 시에서는 조정하지 않아도 이미 동일함)
 * .data == .detach() : 기존 텐서를 복사함
   * 위 계산에서는기존 텐서 y 를 그대로 사용해도 상관은 없음
   * 단, pytorch에서는 .data 보다는 .detach() 를 사용하기를 권장함 [https://subinium.github.io/pytorch-Tensor-Variable/] 

In [19]:
best_val_loss = None
for e in range(1, EPOCHS+1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)

    print("[Epoch: %d] val loss : %5.2f | val accuracy : %5.2f" % (e, val_loss, val_accuracy))

    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("snapshot"):
            os.makedirs("snapshot")
        torch.save(model.state_dict(), './snapshot/txtclassification.pt')
        best_val_loss = val_loss

[Epoch: 1] val loss :  0.70 | val accuracy : 50.44
[Epoch: 2] val loss :  0.69 | val accuracy : 50.36
[Epoch: 3] val loss :  0.71 | val accuracy : 49.40
[Epoch: 4] val loss :  0.49 | val accuracy : 77.52
[Epoch: 5] val loss :  0.35 | val accuracy : 85.50
[Epoch: 6] val loss :  0.34 | val accuracy : 86.02
[Epoch: 7] val loss :  0.38 | val accuracy : 85.72
[Epoch: 8] val loss :  0.42 | val accuracy : 85.98
[Epoch: 9] val loss :  0.50 | val accuracy : 85.46
[Epoch: 10] val loss :  0.49 | val accuracy : 85.98


* ```model.state_dict()```
 * Returns a dictionary containing a whole state of the module. Both parameters and persistent buffers are included.


In [82]:
# model.state_dict() 확인
for key in model.state_dict().keys():
  size = model.state_dict()[key].shape
  print('{} size :'.format(key), list(size))

embed.weight size : [46159, 128]
gru.weight_ih_l0 size : [768, 128]
gru.weight_hh_l0 size : [768, 256]
gru.bias_ih_l0 size : [768]
gru.bias_hh_l0 size : [768]
out.weight size : [2, 256]
out.bias size : [2]
